In [29]:
import torch
from tokenizers import ByteLevelBPETokenizer
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel, GPT2Config
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset

In [5]:
# Pre-trained polish GPT2 model
model = GPT2LMHeadModel.from_pretrained('flax-community/papuGaPT2')
tokenizer = GPT2Tokenizer.from_pretrained('flax-community/papuGaPT2')
tokenizer.pad_token = 0

Downloading:   0%|          | 0.00/864 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/867k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208 [00:00<?, ?B/s]

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def perplexity(input_ids, model):
    length = model.config.n_positions
    nlls = []
    for i in range(0, input_ids.size(1)-length, length):
        curr_input_ids = input_ids[:, i:i+length].to(device)
        with torch.no_grad():
            outputs = model(curr_input_ids, labels=curr_input_ids)
        nlls.append(outputs['loss'])
    return torch.exp(sum(nlls)/len(nlls))

In [6]:
test_dataset = load_dataset('text', data_files='data/test.txt')
tokenized_test_dataset = tokenizer('\n\n'.join(test_dataset['train']['text']), return_tensors="pt")

Using custom data configuration default-5414c941d6230d78
Reusing dataset text (/home/ania/.cache/huggingface/datasets/text/default-5414c941d6230d78/0.0.0/acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
perplexity(tokenized_test_dataset.input_ids, model)

tensor(314.3903)

In [22]:
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    logits = outputs.logits
    output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True, 
                            min_length=30, max_length=50)
    return tokenizer.decode(output['sequences'][0])

In [23]:
generate_text("Kot siedział na drzewie i ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Kot siedział na drzewie i łkał. Gdy się obudził, zobaczył, że w jego pokoju jest kot. Kot zaczął się drapać i mruczeć. Kot zaczął się śmiać. Kot zaczął się śmiać. Kot zaczął się śmiać. Kot zaczął'

In [24]:
generate_text("To nie jest tak, że dobrze albo niedobrze ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'To nie jest tak, że dobrze albo niedobrze rządzenie jest złe. To jest po prostu bardzo dobre.\nW tym roku, w którym obchodzimy 100-lecie odzyskania niepodległości, w wielu miastach Polski odbędą się uroczystości związane z obchodami tego święta.'

In [27]:
def tokenize(row):
    return tokenizer(row['text'])

train_dataset = load_dataset('text', data_files='data/small_unsegmented.txt')
tokenized_train_dataset = train_dataset['train'].map(tokenize, batched=True)

Using custom data configuration default-5414c941d6230d78
Reusing dataset text (/home/ania/.cache/huggingface/datasets/text/default-5414c941d6230d78/0.0.0/acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [31]:
training_args = TrainingArguments(output_dir="gpt_pretrained", save_steps=2000, save_total_limit=1)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_train_dataset, data_collator=data_collator)

In [38]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 20000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7500


Step,Training Loss


KeyboardInterrupt: 

In [39]:
trainer.save_model('gpt_pretrained')

Saving model checkpoint to gpt_pretrained
Configuration saved in gpt_pretrained/config.json
Model weights saved in gpt_pretrained/pytorch_model.bin


In [40]:
model = GPT2LMHeadModel.from_pretrained('gpt_pretrained')

loading configuration file gpt_pretrained/config.json
Model config GPT2Config {
  "_name_or_path": "flax-community/papuGaPT2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.20.0",
  "us

In [42]:
generate_text("Kot siedział na drzewie i ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Kot siedział na drzewie i łkał. W pewnym momencie, gdy był już w drodze, w jego głowie pojawiły się myśli.\n- To nie jest tak, że ja nie wiem, co się stało. Ja wiem, że to był wypadek'

In [43]:
generate_text("To nie jest tak, że dobrze albo niedobrze ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'To nie jest tak, że dobrze albo niedobrze rządzenie jest w stanie zapewnić nam to, co jest w stanie zapewnić nam państwo. To jest państwo, które jest w stanie zapewnić nam państwo, które jest w stanie zapewnić nam państwo, które jest w'

In [45]:
perplexity(tokenized_test_dataset.input_ids, model)

tensor(439.8239)